In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model, load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

#..........................................
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
import os



Using TensorFlow backend.


In [2]:
def SAResNet(input_shape, classes):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((2, 2))(X_input)

    # Stage 1
    X = Conv2D(8, (3, 3), strides=(1, 1), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(1, 1))(X)
    print(X.shape)

    # Stage 2
    # First Residual block
    X = Conv2D(16, (3, 3), strides = (2,2), name = 'conv2', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    print(X.shape)
    
    # Save the input value
    X_shortcut = X

    # Second component of main path (≈3 lines)
    X = Conv2D(8, (1, 1), strides = (1,1), padding = 'same', name = 'conv3', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)
    X = Activation('relu')(X)
    
    print(X.shape)

    # Third component of main path (≈2 lines)
    X = Conv2D(16, (3, 3), strides = (1,1), padding = 'same', name = 'conv4', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv4')(X)

    print(X.shape)
    
    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(16, (1, 1), strides = (1,1), padding = 'same', name = 'res-1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = 'bn_res-1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    print(X.shape)


    # Stage 3 
    # second residual block
    X = Conv2D(32, (3, 3), strides = (2,2), name = 'conv5', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv5')(X)
    X = Activation('relu')(X)
    
    # Save the input value
    X_shortcut = X

    # Second component of main path (≈3 lines)
    X = Conv2D(16, (1, 1), strides = (1,1), padding = 'same', name = 'conv6', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv6')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(32, (3, 3), strides = (1,1), padding = 'same', name = 'conv7', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv7')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(32, (3, 3), strides = (1,1), padding = 'same', name = 'res-2',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = 'bn_res-2')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    print(X.shape)

    # Stage 4 (≈6 lines)
    # residual block 3
    X = Conv2D(64, (3, 3), strides = (2,2), name = 'conv8',padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv8')(X)
    X = Activation('relu')(X)
    
    # Self attention layer
    X = attention(X, channels=64)
    
    # Save the input value
    X_shortcut = X
    
    print (X.shape)

    # Second component of main path (≈3 lines)
    X = Conv2D(32, (1, 1), strides = (1,1), padding = 'same', name = 'conv9', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv9')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(64, (3, 3), strides = (1,1), padding = 'same', name = 'conv10', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv10')(X)
    
    print (X.shape)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(64, (3, 3), strides = (1,1), padding = 'same', name = 'res-3',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = 'bn_res-3')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    print('RB-3: ', X.shape)

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool1")(X)
    
    # final stage
    X = Conv2D(64, (3, 3), strides = (1,1), padding = 'same', name = 'conv11', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv11')(X)
    X = Activation('relu')(X)
    
    print(X.shape)

    X = Conv2D(128, (3, 3), strides = (2,2), padding = 'same', name = 'conv12', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv12')(X)
    
    print(X.shape)

    ##### SHORTCUT PATH #### (≈2 lines)
    X = Conv2D(64, (1, 1), strides = (1,1), padding = 'same', name = 'conv13',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv13')(X)
    
    print(X.shape)

    X = AveragePooling2D((2,2), name="avg_pool2")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='SAResNet')

    return model

In [3]:
def attention(x, channels):
    batch_size, height, width, num_channels = x.get_shape().as_list()
    f = Conv2D(channels // 8, kernel_size=(1,1), strides=(1,1), padding = 'same', use_bias = True)(x) # [bs, h, w, c']
    f = Activation('relu')(f)
    g = Conv2D(channels // 8, kernel_size=(1,1), strides=(1,1), padding = 'same', use_bias = True)(x) # [bs, h, w, c']
    g = Activation('relu')(g)
    h = Conv2D(channels, kernel_size=(1,1), strides=(1,1), padding = 'same', use_bias = True)(x) # [bs, h, w, c]
    h = Activation('relu')(h)
    print(f.shape)
    print(g.shape)
    print(h.shape)
#     f=hw_flatten(f)

    area = height*width
    f = tf.reshape(f,shape = [-1, area, f.shape[-1]])
    g = tf.reshape(g,shape = [-1, area, g.shape[-1]])
    h = tf.reshape(h,shape = [-1, area, h.shape[-1]])
    
    print(f.shape)
#     print(hw_flatten(g).shape)
    

    # N = h * w
#     s = tf.matmul(hw_flatten(f), hw_flatten(g), transpose_b=True) # # [bs, N, N]

    s = tf.matmul(g, f, transpose_b=True)
    beta = tf.keras.activations.softmax(s, axis=-1)  # attention map

    SA = tf.matmul(beta, h) # [bs, N, C]
    gamma = tf.compat.v1.get_variable("gamma", [1], initializer=tf.constant_initializer(0.0))

    SA = tf.reshape(SA, shape = [-1,height, width,num_channels]) # [bs, h, w, C]
    SA = Conv2D(channels, kernel_size=(1,1), strides=(1,1), padding = 'same', name = 'self_attention')(SA)

#     out = gamma * SA + x
    out = Add()([gamma*SA, x])
    print(out.shape)

    return out

In [4]:
model = SAResNet(input_shape = (256, 256, 3), classes = 6)
model.summary()

AttributeError: module 'tensorflow' has no attribute 'random_uniform'

In [5]:
# Compiling the CNN
model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])



In [7]:
# image preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64

train_data_dir = "D:/Anil/Dataset/plant_data/train"     # directory of training data

test_data_dir = "D:/Anil/Dataset/plant_data/val"      # directory of Validation data

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(256, 256),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(test_data_dir,
                                            target_size=(256, 256),
                                            batch_size=batch_size,
                                            class_mode='categorical')

print(training_set.class_indices)



Found 2673 images belonging to 6 classes.
Found 297 images belonging to 6 classes.
{'Apple_black_rot': 0, 'Apple_scab': 1, 'Potato_early_blight': 2, 'Potato_late_blight': 3, 'Strawberry_gray_mold': 4, 'Strawberry_scorch': 5}


In [8]:
# checkpoint
weightpath = "D:/Anil/saved_model/SACNN/SA_ResNet_6_class.hdf5"
checkpointer = ModelCheckpoint(weightpath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

start_time = datetime.datetime.now()
print("start time: "+str(start_time))

#fitting images to CNN
history = model.fit(training_set,
                         steps_per_epoch=training_set.samples//batch_size,
                         validation_data=test_set,
                         epochs=80,
                         validation_steps=test_set.samples//batch_size,
                         callbacks=[checkpointer])#callbacks_list)


# #fitting images to CNN
# history = classifier.fit_generator(training_set,
#                                    steps_per_epoch=training_set.samples//batch_size,
#                                    validation_data=test_set,
#                                    epochs=10,
#                                    validation_steps=test_set.samples//batch_size)

# Stop time
stop_time = datetime.datetime.now()
print("stop time: "+str(stop_time))

total_time = stop_time - start_time
print("Total time: "+str(total_time))

start time: 2021-04-04 14:31:47.432148
Epoch 1/80
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node SAResNet/conv1/Conv2D (defined at <ipython-input-8-7ef61e592599>:14) ]] [Op:__inference_train_function_4448]

Function call stack:
train_function
